In [ ]:
!pip install foolbox

In [ ]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

In [ ]:
from __future__ import print_function
import torch
import numpy as np
import warnings
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
import torchvision.models as models

In [ ]:
class Model_Drop(nn.Module):
    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 600)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)
        self.drop_layer = nn.Dropout(p=0.25)

    def last_hidden_layer_output(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.drop_layer(F.relu(self.conv3(x)))
        x = self.drop_layer(F.relu(self.conv4(x)))
        x = x.view(-1, 7*7*64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x

In [ ]:
def unc_defense(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model((image+delta).clamp(0,1))
            output = softmax(output)

            dropout_predictions[i] = output


        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)


        model.eval()
        o = model((image + delta).clamp(0, 1))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)


        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta).clamp(0, 1))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc == delta_loss, zeros, delta_unc)
            delta.data = (delta - alpha * delta_unc).clamp(-epsilon, epsilon)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image.detach()

In [ ]:
warnings.filterwarnings("ignore")


torch.manual_seed(2)
np.random.seed(2)

batch_size = 64
eps = 0.05
alpha = 0.2 * eps
num_iter = 10
eps_l2 = 0.67

eps_little = 0.001
alpha_reverse = 0.2 * eps_little
num_iter_reverse = 10

count_successful_reverse = 0
count_unsuccessful_reverse = 0

count_successfull_attack = 0
count_unsuccessfull_attack = 0

test_data = datasets.FashionMNIST(root='/FashionData', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_cnn = Model_Drop()
model_cnn.load_state_dict(torch.load("model_cnn.pt",map_location=device))
model_cnn.eval()
model_cnn.to(device)

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

2021-02-23 09:44:15


In [ ]:
for i, (image, label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(image)
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]


    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    attack = L2CarliniWagnerAttack(steps=1000, abort_early=False)
    fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
    raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    pert = torch.tensor(clipped_advs[0])

    #attack = L2DeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps_l2])
    #pert = torch.tensor(clipped_advs[0])

    # pert = image

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(pert)
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]


    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]


    if inds_wrong_after_attack.shape[0] == 0:
        continue

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]
    pred_pert = pred_pert[inds_wrong_after_attack]

    reversed_pert = unc_defense(pert, model_cnn, eps_little, num_iter_reverse, alpha_reverse)

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(reversed_pert)
        o = softmax(o)
    pred_reverse = o.data.max(1, keepdim=True)[1]

    pred_reverse = pred_reverse.view_as(label)

    inds_correct_after_reverse = np.where(pred_reverse.cpu() == label.cpu())[0]
    inds_wrong_after_reverse = np.where(pred_reverse.cpu() != label.cpu())[0]

    reversed_pert = reversed_pert[inds_correct_after_reverse]
    image = image[inds_correct_after_reverse]
    label = label[inds_correct_after_reverse]
    pert = pert[inds_correct_after_reverse]
    pred_pert = pred_pert[inds_correct_after_reverse]


    inds_correct_after_reverse = inds_correct_after_reverse.tolist()
    inds_wrong_after_reverse = inds_wrong_after_reverse.tolist()

    count_successful_reverse += len(inds_correct_after_reverse)
    count_unsuccessful_reverse += len(inds_wrong_after_reverse)

    #if i == 0:
        #break

In [ ]:
print("Number of successful reverse operation is : ", count_successful_reverse)
print("Number of unsuccessful reverse operation is : ", count_unsuccessful_reverse)

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful reverse operation is :  8331
Number of unsuccessful reverse operation is :  19
2021-02-23 11:16:14
